In [1]:
# 필요 라이브러리 호출
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')
from IPython.display import display
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothic'

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# 데이터 불러오기
df = pd.read_csv('./final_grid_data.csv')
df.head()

,Unnamed: 0,gid,geometry,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,patrol_cnt,...,m_80g_pop,w_80g_pop,m_90g_pop,w_90g_pop,m_100_pop,w_100g_pop,total_pop,adult_game_cnt,adult_play_cnt,adult_stay_cnt
0,0,다사49aa19ab,MULTIPOLYGON (((126.9247686191008 37.270765730...,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,1,다사49aa19ba,MULTIPOLYGON (((126.9247514746132 37.273019120...,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,2,다사49aa20aa,MULTIPOLYGON (((126.9247171799136 37.277525898...,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,3,다사49ab18ab,MULTIPOLYGON (((126.9276564748467 37.261765825...,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,4,다사49ab18ba,MULTIPOLYGON (((126.9276394219782 37.264019219...,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [3]:
# 필요 데이터 추출
list_ = ['gid', 'cctv_cnt', 'securitylight_cnt', 'streetlight_cnt', 'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 
         'adult_play_cnt', 'adult_stay_cnt', 'rep_cnt', 'total_pop']
df1 = df[list_]
df1.head()

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop
0,다사49aa19ab,0,0,0,0,0,0,0,0,0,0,0.0
1,다사49aa19ba,0,0,0,0,0,0,0,0,0,0,0.0
2,다사49aa20aa,0,0,0,0,0,0,0,0,0,0,0.0
3,다사49ab18ab,0,0,0,0,0,0,0,0,0,0,0.0
4,다사49ab18ba,0,0,0,0,0,0,0,0,0,0,0.0


### 안전지수 도출 1
- 전체적으로 단위의 통일이 필요. 5개의 레벨로 각 변수들을 점수화

In [4]:
def make_safety_index(df):
    new_df = df[['cctv_cnt', 'securitylight_cnt', 'streetlight_cnt',
               'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 'adult_play_cnt', 'adult_stay_cnt', 'rep_cnt']]
    
    new_df['cctv_range'] = pd.cut(new_df['cctv_cnt'], 5, duplicates='drop', labels=False)
    new_df['sec_range'] = pd.cut(new_df['securitylight_cnt'], 5, duplicates='drop', labels=False)
    new_df['st_range'] = pd.cut(new_df['streetlight_cnt'], 5, duplicates='drop', labels=False)
    new_df['post_range'] = new_df['womanpost_cnt']
    new_df['bell_range'] = new_df['bell_cnt']
    new_df['park_range'] = new_df['park_cnt']
    new_df['game_range'] = new_df['adult_game_cnt']
    new_df['play_range'] = pd.cut(new_df['adult_play_cnt'], 5, duplicates='drop', labels=False)
    new_df['stay_range'] = pd.cut(new_df['adult_stay_cnt'], 5, duplicates='drop', labels=False)
    new_df['rep_range'] = pd.cut(new_df['rep_cnt'], 5, duplicates='drop', labels=False)
    new_df = new_df.astype('int')
    
    new_df['game_range'] = new_df['game_range'] * -1
    new_df['play_range'] = new_df['play_range'] * -1
    new_df['stay_range'] = new_df['stay_range'] * -1
    new_df['rep_range'] = new_df['rep_range'] * -1
    
    sum = new_df[['cctv_range','sec_range','st_range','post_range','bell_range','park_range','game_range','play_range','stay_range'
                 ,'rep_range']].sum(axis=1).tolist()
    
    df['score'] = sum
    df['safety_index'] = pd.cut(df['score'], 6, duplicates='drop',labels=['심각','매우 나쁨','나쁨','양호','좋음','매우 좋음'])

    return df  

In [5]:
make_safety_index(df1)

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop,score,safety_index
0,다사49aa19ab,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
1,다사49aa19ba,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
2,다사49aa20aa,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
3,다사49ab18ab,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
4,다사49ab18ba,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085,다사63ba21aa,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
2086,다사63ba21ab,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
2087,다사63ba21ba,0,0,0,0,0,0,0,0,0,0,0.0,0,나쁨
2088,다사63ba21bb,0,0,2,0,0,0,0,0,0,0,0.0,0,나쁨


In [6]:
df1['safety_index'].value_counts()

나쁨       1631
양호        384
좋음         60
매우 나쁨       9
매우 좋음       5
심각          1
Name: safety_index, dtype: int64

### 안전지수 도출 2
- 안전지수 도출 1 결과, 신고횟수가 0인 격자가 매우좋음으로 나오지 않는 문제가 발생.
  - 안전지수에서 신고횟수의 영향이 다른 변수들에 비해 상대적으로 매우 크게 나타남.
  - 신고횟수를 안전지수 변수에서 제외.
  - 신고횟수가 0인 지역 제외.
  - 인구수가 0인 지역 제외.

- 신고횟수를 제외하였으니, 나머지 변수가 모두 시설물 갯수로 성격이 동일. 


- 범주형 변수를 제외한 나머지 변수에 대해 pd.cut을 이용하여 10개 단위로 점수화

In [8]:
list2 = ['gid', 'cctv_cnt', 'securitylight_cnt', 'streetlight_cnt', 'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 
         'adult_play_cnt', 'adult_stay_cnt','rep_cnt', 'total_pop']
df2 = df[list2][(df['rep_cnt'] != 0) & df['total_pop'] != 0]
df2

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop
21,다사49ba19ba,0,0,5,0,0,0,0,0,0,11,198.0
22,다사49ba19bb,2,0,5,0,0,0,0,0,0,12,336.0
39,다사49bb19ab,4,11,9,0,0,0,0,0,0,25,219.0
40,다사49bb19ba,5,0,23,0,0,0,0,0,0,118,2359.0
41,다사49bb19bb,5,0,11,0,0,0,0,0,0,35,726.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2064,다사63aa18ab,9,0,7,0,0,1,0,0,0,76,1233.0
2065,다사63aa18ba,5,0,8,0,0,1,0,0,0,86,2475.0
2066,다사63aa18bb,6,0,9,0,0,1,0,0,0,103,2100.0
2072,다사63aa21bb,3,0,16,0,0,0,0,0,0,14,1604.0


In [9]:
def make_safety_index2(df):
    new_df = df[['cctv_cnt', 'securitylight_cnt', 'streetlight_cnt',
               'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 'adult_play_cnt', 'adult_stay_cnt']]
    
    new_df['cctv_range'] = pd.cut(new_df['cctv_cnt'], 5, duplicates='drop', labels=False)
    new_df['sec_range'] = pd.cut(new_df['securitylight_cnt'], 14, duplicates='drop', labels=False)
    new_df['st_range'] = pd.cut(new_df['streetlight_cnt'], 13, duplicates='drop', labels=False)
    new_df['post_range'] = new_df['womanpost_cnt']
    new_df['bell_range'] = new_df['bell_cnt']
    new_df['park_range'] = new_df['park_cnt']
    new_df['game_range'] = new_df['adult_game_cnt']
    new_df['play_range'] = pd.cut(new_df['adult_play_cnt'], 2, duplicates='drop', labels=False)
    new_df['stay_range'] = pd.cut(new_df['adult_stay_cnt'], 3, duplicates='drop', labels=False)
    new_df = new_df.astype('int')
    
    new_df['game_range'] = new_df['game_range'] * -1
    new_df['play_range'] = new_df['play_range'] * -1
    new_df['stay_range'] = new_df['stay_range'] * -1
    
    sum = new_df[['cctv_range','sec_range','st_range','post_range','bell_range','park_range','game_range','play_range','stay_range'
                 ]].sum(axis=1).tolist()
    
    df['score'] = sum
    df['safety_index'] = pd.cut(df['score'], 6, duplicates='drop',labels=['심각','매우 나쁨','나쁨','양호','좋음','매우 좋음'])

    return df

In [10]:
make_safety_index2(df2)

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop,score,safety_index
21,다사49ba19ba,0,0,5,0,0,0,0,0,0,11,198.0,0,심각
22,다사49ba19bb,2,0,5,0,0,0,0,0,0,12,336.0,0,심각
39,다사49bb19ab,4,11,9,0,0,0,0,0,0,25,219.0,1,심각
40,다사49bb19ba,5,0,23,0,0,0,0,0,0,118,2359.0,2,심각
41,다사49bb19bb,5,0,11,0,0,0,0,0,0,35,726.0,1,심각
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,다사63aa18ab,9,0,7,0,0,1,0,0,0,76,1233.0,1,심각
2065,다사63aa18ba,5,0,8,0,0,1,0,0,0,86,2475.0,1,심각
2066,다사63aa18bb,6,0,9,0,0,1,0,0,0,103,2100.0,1,심각
2072,다사63aa21bb,3,0,16,0,0,0,0,0,0,14,1604.0,1,심각


In [11]:
df2['safety_index'].value_counts()

심각       573
매우 나쁨    292
나쁨       105
양호        28
좋음         4
매우 좋음      1
Name: safety_index, dtype: int64

### 안전지수 도출 3
- 변수 특징을 살펴보고 cut 대신 qcut을 사용

In [14]:
list3 = ['gid', 'cctv_cnt', 'securitylight_cnt', 'streetlight_cnt', 'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 
         'adult_play_cnt', 'adult_stay_cnt','rep_cnt', 'total_pop']
df3 = df[list3][(df['rep_cnt'] != 0) & df['total_pop'] != 0]
df3

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop
21,다사49ba19ba,0,0,5,0,0,0,0,0,0,11,198.0
22,다사49ba19bb,2,0,5,0,0,0,0,0,0,12,336.0
39,다사49bb19ab,4,11,9,0,0,0,0,0,0,25,219.0
40,다사49bb19ba,5,0,23,0,0,0,0,0,0,118,2359.0
41,다사49bb19bb,5,0,11,0,0,0,0,0,0,35,726.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2064,다사63aa18ab,9,0,7,0,0,1,0,0,0,76,1233.0
2065,다사63aa18ba,5,0,8,0,0,1,0,0,0,86,2475.0
2066,다사63aa18bb,6,0,9,0,0,1,0,0,0,103,2100.0
2072,다사63aa21bb,3,0,16,0,0,0,0,0,0,14,1604.0


In [15]:
df3.describe()

,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop
count,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000,1003.000000
mean,10.717846,13.453639,19.039880,0.015952,0.140578,0.298106,0.015952,0.494516,0.272183,211.203390,971.643071
std,9.333430,17.504849,14.209384,0.125353,0.476157,0.561459,0.147314,1.405265,1.456863,345.547788,721.028222
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000
25%,4.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000,370.500000
50%,9.000000,5.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,118.000000,871.000000
75%,16.000000,22.500000,26.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,245.500000,1458.000000
max,50.000000,134.000000,130.000000,1.000000,4.000000,4.000000,3.000000,19.000000,23.000000,4729.000000,4923.000000


In [35]:
cv = []

for i in range(1,9):
    cv.append(df3.columns[i])
    cv.append((df3.iloc[:,i].std()) / (df3.iloc[:,i].mean()))

cv

['cctv_cnt',
 0.8708307703058011,
 'securitylight_cnt',
 1.3011237595211005,
 'streetlight_cnt',
 0.7462958621459126,
 'womanpost_cnt',
 7.858057289894024,
 'bell_cnt',
 3.387127458835057,
 'park_cnt',
 1.883421829875054,
 'adult_game_cnt',
 9.234735392093206,
 'adult_play_cnt',
 2.8416955682708145]

- 변수들의 변동계수 최소값이 74%이며 df3.describe()를 보면 3사분위수와 최댓값 차이가 상당한 것을 알 수 있다.
- 따라서 10개씩 동일한 길이로 점수화하지 않고 동일한 개수로 나누어 0~4점을 부여한다.

In [36]:
def make_safety_index3(df):
    new_df = df[['cctv_cnt', 'securitylight_cnt', 'streetlight_cnt',
               'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 'adult_play_cnt', 'adult_stay_cnt']]
    
    new_df['cctv_range'] = pd.qcut(new_df['cctv_cnt'], 5, duplicates='drop', labels=False)
    new_df['sec_range'] = pd.qcut(new_df['securitylight_cnt'], 5, duplicates='drop', labels=False)
    new_df['st_range'] = pd.qcut(new_df['streetlight_cnt'], 5, duplicates='drop', labels=False)
    new_df['post_range'] = new_df['womanpost_cnt']
    new_df['bell_range'] = new_df['bell_cnt']
    new_df['park_range'] = new_df['park_cnt']
    new_df['game_range'] = new_df['adult_game_cnt']
    new_df['play_range'] = pd.qcut(new_df['adult_play_cnt'], 5, duplicates='drop', labels=False)
    new_df['stay_range'] = pd.qcut(new_df['adult_stay_cnt'], 5, duplicates='drop', labels=False)
    new_df = new_df.astype('int')
    
    new_df['game_range'] = new_df['game_range'] * -1
    new_df['play_range'] = new_df['play_range'] * -1
    new_df['stay_range'] = new_df['stay_range'] * -1
    
    sum = new_df[['cctv_range','sec_range','st_range','post_range','bell_range','park_range','game_range','play_range','stay_range'
                 ]].sum(axis=1).tolist()
    
    df['score'] = sum
    df['safety_index'] = pd.cut(df['score'], 6, duplicates='drop',labels=['심각','매우 나쁨','나쁨','양호','좋음','매우 좋음'])

    print(df['safety_index'].value_counts())
    return df

In [43]:
df_index = make_safety_index3(df3)

나쁨       372
매우 나쁨    205
심각       181
양호       178
좋음        60
매우 좋음      7
Name: safety_index, dtype: int64


### 안전지수 도출 4
- 신고횟수가 245 (3사분위수) 이상인 지역에 대해서 필터링 후 안전지수 도출

In [56]:
list4 = ['gid', 'cctv_cnt', 'securitylight_cnt', 'streetlight_cnt', 'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 
         'adult_play_cnt', 'adult_stay_cnt','rep_cnt', 'total_pop']
df4 = df[list4][(df['rep_cnt'] > 245) & df['total_pop'] != 0]
df4

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop
103,다사50ba19ba,6,0,33,0,0,0,0,0,0,285,538.0
111,다사50ba21ba,12,8,20,0,0,1,0,1,0,499,65.0
170,다사51ab19ba,3,0,42,0,0,0,0,2,0,1757,472.0
195,다사51ba19ba,14,0,46,0,0,0,0,1,0,555,522.0
248,다사52aa17bb,1,0,21,0,0,0,0,0,0,357,2071.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2027,다사62ba16ba,9,0,30,0,0,0,0,2,0,422,345.0
2028,다사62ba16bb,23,0,37,0,0,2,0,0,0,433,650.0
2029,다사62ba17aa,14,0,35,0,0,0,0,2,0,456,1195.0
2030,다사62ba17ab,2,0,27,0,0,0,0,1,0,2237,1960.0


In [59]:
def make_safety_index4(df):
    new_df = df[['cctv_cnt', 'securitylight_cnt', 'streetlight_cnt',
               'womanpost_cnt', 'bell_cnt', 'park_cnt', 'adult_game_cnt', 'adult_play_cnt', 'adult_stay_cnt']]
    
    new_df['cctv_range'] = pd.qcut(new_df['cctv_cnt'], 5, duplicates='drop', labels=False)
    new_df['sec_range'] = pd.qcut(new_df['securitylight_cnt'], 5, duplicates='drop', labels=False)
    new_df['st_range'] = pd.qcut(new_df['streetlight_cnt'], 5, duplicates='drop', labels=False)
    new_df['post_range'] = new_df['womanpost_cnt']
    new_df['bell_range'] = new_df['bell_cnt']
    new_df['park_range'] = new_df['park_cnt']
    new_df['game_range'] = new_df['adult_game_cnt']
    new_df['play_range'] = pd.qcut(new_df['adult_play_cnt'], 5, duplicates='drop', labels=False)
    new_df['stay_range'] = pd.qcut(new_df['adult_stay_cnt'], 5, duplicates='drop', labels=False)
    new_df = new_df.astype('int')
    
    new_df['game_range'] = new_df['game_range'] * -1
    new_df['play_range'] = new_df['play_range'] * -1
    new_df['stay_range'] = new_df['stay_range'] * -1
    
    sum = new_df[['cctv_range','sec_range','st_range','post_range','bell_range','park_range','game_range','play_range','stay_range'
                 ]].sum(axis=1).tolist()
    
    df['score'] = sum
    df['safety_index'] = pd.qcut(df['score'], 6, duplicates='drop',labels=['심각','매우 나쁨','나쁨','양호','좋음','매우 좋음'])

    print(df['safety_index'].value_counts())
    return df

In [68]:
df_index2 = make_safety_index4(df4)

나쁨       65
심각       54
매우 좋음    40
양호       35
매우 나쁨    32
좋음       25
Name: safety_index, dtype: int64


In [74]:
df_index2[df_index2['safety_index'] == '심각'].reset_index(drop=True).describe()

,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop,score
count,54.000000,54.000000,54.000000,54.0,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000,54.000000
mean,8.500000,12.240741,18.333333,0.0,0.037037,0.092593,0.148148,2.277778,1.277778,745.037037,1053.759259,2.092593
std,4.855458,13.058555,9.009429,0.0,0.190626,0.292582,0.491720,3.394872,2.513486,715.612443,750.029784,1.169882
min,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,246.000000,147.000000,-1.000000
25%,5.000000,0.000000,12.250000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,293.500000,454.750000,2.000000
50%,9.000000,8.500000,17.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,421.000000,837.500000,2.000000
75%,11.000000,18.750000,23.000000,0.0,0.000000,0.000000,0.000000,3.000000,1.000000,893.750000,1605.500000,3.000000
max,23.000000,47.000000,42.000000,0.0,1.000000,1.000000,3.000000,19.000000,12.000000,3447.000000,2935.000000,3.000000


In [73]:
df_index2[df_index2['safety_index'] == '매우 나쁨'].reset_index(drop=True).describe()

,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop,score
count,32.000000,32.000000,32.000000,32.0,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.0
mean,12.437500,18.437500,20.156250,0.0,0.093750,0.125000,0.031250,1.406250,1.906250,684.312500,1114.343750,4.0
std,7.002016,15.261035,10.653212,0.0,0.296145,0.336011,0.176777,1.521022,4.252964,920.317072,699.262082,0.0
min,3.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,249.000000,21.000000,4.0
25%,8.750000,7.500000,14.500000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,298.500000,559.000000,4.0
50%,11.500000,16.000000,18.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,368.000000,1082.500000,4.0
75%,14.000000,30.000000,24.000000,0.0,0.000000,0.000000,0.000000,2.000000,1.000000,497.500000,1655.250000,4.0
max,37.000000,61.000000,49.000000,0.0,1.000000,1.000000,1.000000,7.000000,16.000000,4729.000000,2458.000000,4.0


### 후보지 필터링

- Safety Index가 '심각'인 것을 대상으로 후보지를 필터링

In [44]:
df_index[df_index['safety_index'] == '심각'].reset_index(drop=True)

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop,score,safety_index
0,다사49ba19ba,0,0,5,0,0,0,0,0,0,11,198.0,0,심각
1,다사49ba19bb,2,0,5,0,0,0,0,0,0,12,336.0,0,심각
2,다사49bb19bb,5,0,11,0,0,0,0,0,0,35,726.0,2,심각
3,다사49bb20ba,0,7,0,0,0,0,0,0,0,4,6.0,1,심각
4,다사50aa19ab,0,0,18,0,0,0,0,0,0,101,2897.0,2,심각
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,다사62bb18ab,0,0,17,0,0,0,0,0,0,98,2221.0,2,심각
177,다사62bb18bb,4,0,9,0,0,0,0,0,0,78,354.0,2,심각
178,다사62bb21bb,2,0,4,0,0,0,0,0,0,10,971.0,0,심각
179,다사63aa21bb,3,0,16,0,0,0,0,0,0,14,1604.0,2,심각


In [89]:
df_index2[df_index2['safety_index'] == '심각'].reset_index(drop=True)

,gid,cctv_cnt,securitylight_cnt,streetlight_cnt,womanpost_cnt,bell_cnt,park_cnt,adult_game_cnt,adult_play_cnt,adult_stay_cnt,rep_cnt,total_pop,score,safety_index
0,다사51ab19ba,3,0,42,0,0,0,0,2,0,1757,472.0,3,심각
1,다사52aa17bb,1,0,21,0,0,0,0,0,0,357,2071.0,2,심각
2,다사53aa20ba,9,25,23,0,0,0,0,4,0,278,378.0,3,심각
3,다사53aa22ab,9,25,40,0,0,0,1,7,1,2017,449.0,3,심각
4,다사53ba19aa,7,44,14,0,0,0,0,4,0,290,1789.0,3,심각
5,다사54aa17aa,2,42,21,0,0,0,0,4,1,298,834.0,3,심각
6,다사54bb23aa,16,42,16,0,0,0,1,5,0,319,209.0,3,심각
7,다사55aa22aa,10,0,21,0,0,0,0,3,0,360,818.0,2,심각
8,다사55aa22ba,9,14,8,0,0,0,0,1,0,268,2935.0,1,심각
9,다사55aa22bb,10,7,16,0,0,0,0,0,0,255,2665.0,2,심각


### 후보지 데이터프레임 생성

In [75]:
df_final = df_index[df_index['safety_index'] == '심각'].reset_index(drop=True)
df_final2 = df_index2[df_index2['safety_index'] == '심각'].reset_index(drop=True)

In [76]:
df_final = df_final[['gid']]
df_final2 = df_final2[['gid']]

In [77]:
df_final['gid'].tolist()
df_final2['gid'].tolist()

['다사51ab19ba',
 '다사52aa17bb',
 '다사53aa20ba',
 '다사53aa22ab',
 '다사53ba19aa',
 '다사54aa17aa',
 '다사54bb23aa',
 '다사55aa22aa',
 '다사55aa22ba',
 '다사55aa22bb',
 '다사55ba18bb',
 '다사56aa22bb',
 '다사56ab22bb',
 '다사56bb16ba',
 '다사56bb19aa',
 '다사56bb21ab',
 '다사57aa17ab',
 '다사57aa19ba',
 '다사57aa19bb',
 '다사57aa20ab',
 '다사57ba19ba',
 '다사57bb17ba',
 '다사57bb18aa',
 '다사58aa15ab',
 '다사58aa17bb',
 '다사58aa18ab',
 '다사58aa18ba',
 '다사58aa19ba',
 '다사58aa19bb',
 '다사58ab18ab',
 '다사58ab18ba',
 '다사58ab18bb',
 '다사58ab21ba',
 '다사58ba18aa',
 '다사58ba18ab',
 '다사58ba18bb',
 '다사58ba20aa',
 '다사58bb18ba',
 '다사58bb21ba',
 '다사59aa18ba',
 '다사59ab18ba',
 '다사59ab18bb',
 '다사59ab19ba',
 '다사59ba17bb',
 '다사59ba20aa',
 '다사59bb19bb',
 '다사59bb21aa',
 '다사60bb16ab',
 '다사61aa16ba',
 '다사61bb17aa',
 '다사62ab17aa',
 '다사62ba16ba',
 '다사62ba17ab',
 '다사62bb18ba']

In [78]:
df_geo = gpd.read_file('32.수원시_표준격자(250M_x_250M).geojson')
df_geo

,gid,geometry
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ..."
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ..."
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ..."
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ..."
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ..."
...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ..."
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ..."
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ..."
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ..."


In [79]:
geometry = []
for i in range(df_geo.shape[0]):
    if df_geo.iloc[i]['gid'] in df_final['gid'].tolist():
        geometry.append(df_geo.iloc[i]['geometry'])

geometry

In [80]:
geometry2 = []
for i in range(df_geo.shape[0]):
    if df_geo.iloc[i]['gid'] in df_final2['gid'].tolist():
        geometry2.append(df_geo.iloc[i]['geometry'])

geometry2

In [81]:
len(geometry)

181

In [83]:
len(geometry2)

54

In [84]:
df_final['geometry'] = geometry
df_final2['geometry'] = geometry2

In [85]:
df_final

,gid,geometry
0,다사49ba19ba,(POLYGON ((126.9303909103332 37.27304639029154...
1,다사49ba19bb,(POLYGON ((126.9303739320096 37.27529978157728...
2,다사49bb19bb,(POLYGON ((126.9331937367447 37.27531331664526...
3,다사49bb20ba,(POLYGON ((126.9331430426007 37.28207348859672...
4,다사50aa19ab,(POLYGON ((126.9360471619907 37.27081999673347...
...,...,...
176,다사62bb18ab,(POLYGON ((127.0799010786341 37.26240392286228...
177,다사62bb18bb,"(POLYGON ((127.079876042935 37.26691081454412,..."
178,다사62bb21bb,(POLYGON ((127.0797257116635 37.29395209210009...
179,다사63aa21bb,(POLYGON ((127.0825462758711 37.29396207386706...


In [86]:
df_final2

,gid,geometry
0,다사51ab19ba,(POLYGON ((126.9501289943768 37.27313971778609...
1,다사52aa17bb,"(POLYGON ((126.958701116179 37.2574048695601, ..."
2,다사53aa20ba,(POLYGON ((126.9698039500577 37.28224337399134...
3,다사53aa22ab,(POLYGON ((126.9696932512853 37.29801717467198...
4,다사53ba19aa,"(POLYGON ((126.975537924089 37.26874805593837,..."
5,다사54aa17aa,(POLYGON ((126.9813007246617 37.25074557696352...
6,다사54bb23aa,(POLYGON ((126.9893922839988 37.30486420651101...
7,다사55aa22aa,(POLYGON ((126.9922738150387 37.29586272703022...
8,다사55aa22ba,"(POLYGON ((126.992243520709 37.3003695372114, ..."
9,다사55aa22bb,(POLYGON ((126.9922283710153 37.30262294100397...


In [87]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link(df, title = "Download CSV file", filename = "worst.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_final)

In [88]:
create_download_link(df_final2)